In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
train = pd.read_csv('/content/drive/MyDrive/sd솔데스크/추천시스템/train.tsv', sep = '\t')

In [6]:
train

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...


# [중] 대/중/소 형식으로 개별 컬럼을 생성하시오.
category name 을 대/중/소 형식으로 개별 컬럼으로 생성하고 중복을 제거 한 다음 코드를 첨부하시오

In [7]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

def split_대중소(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Other_Null' , 'Other_Null' , 'Other_Null']

train['category_list'] = train['category_name'].apply(lambda x:split_대중소(x))

train['대'] = train['category_list'].apply(lambda x:x[0])
train['중'] = train['category_list'].apply(lambda x:x[1])
train['소'] = train['category_list'].apply(lambda X:X[2])

print(train[['대','중','소']])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                         대                    중                    소
0                      Men                 Tops             T-shirts
1              Electronics  Computers & Tablets   Components & Parts
2                    Women       Tops & Blouses               Blouse
3                     Home           Home Décor   Home Décor Accents
4                    Women              Jewelry            Necklaces
...                    ...                  ...                  ...
1482530              Women              Dresses             Mid-Calf
1482531               Kids          Girls 2T-5T              Dresses
1482532  Sports & Outdoors             Exercise  Fitness accessories
1482533               Home           Home Décor   Home Décor Accents
1482534              Women  Women's Accessories              Wallets

[1482535 rows x 3 columns]


# 2. [중]텍스트를 tokenize를 하시오.

In [8]:
train['word_tokens'] = train['name'].apply(lambda x:x.split())

In [9]:
train

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_list,대,중,소,word_tokens
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,"[Men, Tops, T-shirts]",Men,Tops,T-shirts,"[MLB, Cincinnati, Reds, T, Shirt, Size, XL]"
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,"[Electronics, Computers & Tablets, Components ...",Electronics,Computers & Tablets,Components & Parts,"[Razer, BlackWidow, Chroma, Keyboard]"
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,"[Women, Tops & Blouses, Blouse]",Women,Tops & Blouses,Blouse,"[AVA-VIV, Blouse]"
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,"[Home, Home Décor, Home Décor Accents]",Home,Home Décor,Home Décor Accents,"[Leather, Horse, Statues]"
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,"[Women, Jewelry, Necklaces]",Women,Jewelry,Necklaces,"[24K, GOLD, plated, rose]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...","[Women, Dresses, Mid-Calf]",Women,Dresses,Mid-Calf,"[Free, People, Inspired, Dress]"
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,"[Kids, Girls 2T-5T, Dresses]",Kids,Girls 2T-5T,Dresses,"[Little, mermaid, handmade, dress]"
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.","[Sports & Outdoors, Exercise, Fitness accessor...",Sports & Outdoors,Exercise,Fitness accessories,"[21, day, fix, containers, and, eating, plan]"
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,"[Home, Home Décor, Home Décor Accents]",Home,Home Décor,Home Décor Accents,"[World, markets, lanterns]"


# 3. category_name 변수에서 가장 많이 있는 품목을 코드로 밝히시오(plotly를 사용하여서)

In [10]:
import plotly.express as px

wide_df = train['category_name'].value_counts()[:10]

fig = px.bar(wide_df, x = wide_df.index, y = wide_df.values, title = 'plotly')
fig.show()

# 가장 많은 것에 마우스를 갖다대면 된다

# 4. TF-IDF를 사용하시오.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
train = train.fillna('Others')
tfidf = TfidfVectorizer(stop_words = 'english')
word_token_대 = tfidf.fit_transform(train['대'])

tfidf = TfidfVectorizer(stop_words = 'english')
word_token_중 = tfidf.fit_transform(train['중'])

# 5. 모델을 적용해보고 결과를 해석하시오.

In [ ]:
# 대,중,소를 나눈것을 모델에 넣기 위해서 전처리를 해줬다.
word_token_대 = pd.DataFrame(word_token_대.toarray())
word_token_중 = pd.DataFrame(word_token_중.toarray())
data = pd.concat([word_token_대, word_token_중],axis = 1 )

In [19]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import train_test_split

# rmsle를 사용하여 모델을 평가해보자.
def rmsle(y , y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

# 데이터를 train과 test셋으로 나누었다.
X_train,X_test, y_train, y_test = train_test_split(data, train['price'], test_size = 0.2, random_state = 42)

# 모델 :lightgbm -> 0.764
model = LGBMRegressor(random_state = 42)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print('Lightgbm :',round(rmsle(y_test,y_pred),3))

# 모델 : LinearRegression -> 0.766
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print('LinearRegression :',round(rmsle(y_test,y_pred),3))

# 모델 : Ridge -> 0.766
model = Ridge()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Ridge :',round(rmsle(y_test,y_pred),3))

# 모델 : Lasso -> 0.824
model = Lasso()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Lasso :',round(rmsle(y_test,y_pred),3))

# rmsle는 적게나올수록 좋은 모델이기 때문에 lightgbm이 제일 좋다고 말할 수 있다.

Lightgbm : 0.764
LinearRegression : 0.766
Ridge : 0.766
Lasso : 0.824
